In [61]:
import yaml

yaml_file= "key.yaml"

with open(yaml_file,'r') as f:
    key_info= yaml.safe_load(f)
    openai_key= key_info['OPENAI_API_KEY']

In [62]:
import openai

openai.api_key= openai_key

In [63]:
model= 'gpt-3.5-turbo'

In [64]:
def get_response(messages,model="gpt-3.5-turbo"):

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages)

    return response.choices[0].message.content



In [25]:
import json

def read_lines_from_qbank(file_path,n):
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if i == n:
                try:
                    json_data = json.loads(line.strip())
                    question= json_data['question']+ "\n\n"
                    question+="Options: \n"
                    for key, value in json_data['options'].items():
                        question+=f"{key}. {value}\n"
                    answer= json_data['answer']
                    return question, answer
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON on line {n + 1}: {e}")
                    return None
        print(f"Error: Line {n + 1} not found in {file_path}")
        return None


In [39]:
json_file= './data_clean/questions/US/US_qbank.jsonl'

first_question, first_answer= read_lines_from_qbank(json_file,0)
second_question, second_answer= read_lines_from_qbank(json_file,1)
third_question, third_answer= read_lines_from_qbank(json_file,2)
fourth_question, fourth_answer= read_lines_from_qbank(json_file,3) 

In [42]:
print(first_question, first_answer,  "\n")
print(second_question, second_answer,"\n")
print(third_question, third_answer,"\n")
print(fourth_question, fourth_answer)

A 4670-g (10-lb 5-oz) male newborn is delivered at term to a 26-year-old woman after prolonged labor. Apgar scores are 9 and 9 at 1 and 5 minutes. Examination in the delivery room shows swelling, tenderness, and crepitus over the left clavicle. There is decreased movement of the left upper extremity. Movement of the hands and wrists are normal. A grasping reflex is normal in both hands. An asymmetric Moro reflex is present. The remainder of the examination shows no abnormalities and an anteroposterior x-ray confirms the diagnosis. Which of the following is the most appropriate next step in management?

Options: 
A. Nerve conduction study
B. Surgical fixation
C. Physical therapy
D. Pin sleeve to the shirt
E. Splinting of the arm
F. MRI of the clavicle
 D 

A 66-year-old man comes to the physician for a 3-month history of fatigue. He has hypertension and hyperlipidemia. He had a transient ischemic attack 3 years ago. He drinks 3 beers a day, and sometimes a couple more on social occasion

In [43]:
system_role = "You are a helpful assistant that answers multiple choice questions about medical knowledge."

### Few Shot Prompting

In [65]:
#We give answers to first three question and then ask the answer for the fourth question
user_prompt = f'''Question:{first_question}
Answer:{first_answer}

Question:{second_question}
Answer:{second_answer}

Question:{third_question}
Answer:{third_answer}

Question:{fourth_question}
Answer:
'''

In [66]:
messages=[{'role':'system','content':system_role},
          {'role':'user','content':user_prompt}
          ]

In [48]:
print(get_response(messages)) #gpt 3.5

E. Perform emergency laparotomy


In [67]:
print(get_response(messages=messages, model='gpt-4-turbo')) # gpt 4

E. Perform emergency laparotomy

In emergency situations where a life-threatening condition exists and immediate medical care is necessary, it's legally acceptable to proceed with surgical intervention without explicit parental or guardian consent if they cannot be reached in time. This principle applies here with the critical condition indicating an acute surgical abdomen, likely requiring immediate action to prevent serious harm or death. The law provides for implied consent in such emergencies, especially when dealing with patients who are minors and unable to consent for themselves.


In [49]:
fourth_answer

'E'

Through few shot prompting we could get the correct response to the fourth question.

### One Shot Prompting

In [68]:
user_prompt = f'''Question:{first_question}
Answer:{first_answer}

Question:{fourth_question}
Answer:
'''

In [69]:
messages=[{'role':'system','content':system_role},
          {'role':'user','content':user_prompt}
          ]

In [58]:
print(get_response(messages)) # gpt 3.5

Answer: E. Perform emergency laparotomy

Explanation: In situations where the parents cannot be reached and there is an urgent need for a surgical procedure to save the child's life, the healthcare provider is allowed to proceed with the surgery without parental consent as long as the procedure is necessary to prevent harm. Delaying surgery could result in significant harm or even death to the child. It is crucial to prioritize the well-being and safety of the child in such critical situations.


In [70]:
print(get_response(messages=messages, model='gpt-4-turbo')) # gpt 4

E. Perform emergency laparotomy

In this scenario, the child is experiencing symptoms suggestive of a surgical emergency, such as intestinal obstruction or a similar acute condition characterized by severe abdominal pain, bilious vomiting, abdominal distension, and peritoneal signs. When a minor requires urgent surgical intervention and parents or legal guardians are not available to give consent, medical providers are ethically and legally permitted to proceed with necessary treatments to preserve life or prevent serious harm. This is based on the doctrine of implied consent in emergency situations, where it is assumed that a reasonable person would consent to emergency medical treatment if able. Delaying surgery could potentially worsen the outcome for the patient, making it necessary to perform the emergency laparotomy without delay.


One shot Prompting also provided the correct response

### Zero Shot Prompting

In [71]:
user_prompt = f'''Question:{fourth_question}
Answer:
'''

In [72]:
messages=[{'role':'system','content':system_role},
          {'role':'user','content':user_prompt}
          ]

In [60]:
print(get_response(messages)) # gpt 3.5

Answer: E. Perform emergency laparotomy

In emergency situations where a delay in treatment can lead to serious harm or death, consent can be obtained from a surrogate decision maker if the parents are unavailable. In this case, the patient's brother can be considered as the surrogate decision maker. It is crucial to proceed with the necessary medical intervention promptly to address the acute condition.


In [73]:
print(get_response(messages=messages, model='gpt-4-turbo')) # gpt 4

E. Perform emergency laparotomy

In situations where immediate and urgent action is required to save a patient’s life or prevent serious harm, physicians can proceed with necessary medical interventions such as surgery without consent to save the child's life. The symptoms and findings in this case (severe abdominal pain, bilious vomiting, distended abdomen, signs of peritonitis) suggest an acute surgical abdomen possibly due to conditions like intestinal volvulus or intussusception that require immediate surgical intervention. If the consent of the parents or legal guardians cannot be obtained in a timely manner, and delaying treatment might endanger the patient's life, the surgeon is ethically and legally justified to perform the surgery under the doctrine of implied consent for emergency care.



Zero Shot prompting also provided correct responses